Notebook to train and export tree classifier.

In [ ]:
!pip install -Uqq fastai
!pip install nbdev
from fastai.vision.all import *
!pip install -Uqq duckduckgo_search
from duckduckgo_search import ddg_images
from fastcore.all import *
from fastdownload import download_url
from fastai.vision.all import *
from time import sleep
import shutil
from fastai.vision.widgets import *

Lets find all the images we need.


In [ ]:
#What types of plant are we interested in?
plant_list = ['oak','pine']

In [ ]:
#function to find the images
def search_images(term, max_images=200): return L(ddg_images(term, max_results=max_images)).itemgot('image')

In [ ]:
#where I am going to save the plant photos
path = Path('trees')


for i in plant_list:
    #make the directory
    dest = (path/i)
    
    #if you want to delete a directory
#     shutil.rmtree(dest)

    #download the images
    if dest.is_dir():
        print(f'{dest} already exists, skipping...')
    else:
        dest.mkdir(exist_ok=true,parents=True)
        #download the images
        download_images(dest, urls=search_images(f'{i} tree leaf',max_images=75))
        sleep(10)
        download_images(dest, urls=search_images(f'{i} tree small branches with leaves',max_images=75))
        sleep(10)

    print(f'{len(get_image_files(dest))} image files in {dest}')
    

The correct number does not get downloaded, try DDGS().images instead

In [ ]:
#find the images that can't be opened
failed = verify_images(get_image_files(path))
#apply Path.unlink to every item in Path
failed.map(Path.unlink)

Now Create Data Loader

In [ ]:
trees = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, #will pass the path as an input variable to the datablock object
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)) #this will crop the image

dls_tree = trees.dataloaders(path)

In [ ]:
dls_tree.valid.show_batch(max_n = 5, nrows = 1)

Train Model

In [ ]:
#create learner
learn = vision_learner(dls_tree, resnet18, metrics=error_rate)
learn.fine_tune(4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()

In [ ]:
interp.plot_top_losses(10, nrows = 3, figsize = (20,10))

In [ ]:
cleaner = ImageClassifierCleaner(learn)
cleaner

In [ ]:
#clean the data
for idx in cleaner.delete(): cleaner.fns[idx].unlink()
for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cat)

In [ ]:
learn.export('model.pkl')